In [22]:
from transformers import ViTImageProcessor

processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")

loading configuration file preprocessor_config.json from cache at /home/jialiangfan/.cache/huggingface/hub/models--google--vit-base-patch16-224-in21k/snapshots/b4569560a39a0f1af58e3ddaf17facf20ab919b0/preprocessor_config.json
size should be a dictionary on of the following set of keys: ({'height', 'width'}, {'shortest_edge'}, {'shortest_edge', 'longest_edge'}, {'longest_edge'}), got 224. Converted to {'height': 224, 'width': 224}.
Image processor ViTImageProcessor {
  "_valid_processor_keys": [
    "images",
    "do_resize",
    "size",
    "resample",
    "do_rescale",
    "rescale_factor",
    "do_normalize",
    "image_mean",
    "image_std",
    "return_tensors",
    "data_format",
    "input_data_format"
  ],
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTImageProcessor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size

In [33]:
from torchvision.transforms import (CenterCrop, 
                                    Compose, 
                                    Normalize,
                                    RandomRotation,
                                    RandomResizedCrop,
                                    RandomHorizontalFlip,
                                    RandomAdjustSharpness,
                                    Resize, 
                                    ToTensor)

image_mean, image_std = processor.image_mean, processor.image_std
height = processor.size["height"]
width = processor.size["width"]
size = (height, width)
print("Size: ", size)
print("Image mean: ", image_mean)
print("Image std: ", image_std)

normalize = Normalize(mean=image_mean, std=image_std)
_train_transforms = Compose(
        [
            Resize(size),
            RandomRotation(15),
            RandomAdjustSharpness(2),
            ToTensor(),
            normalize,
        ]
    )

_val_transforms = Compose(
        [
            Resize(size),
            ToTensor(),
            normalize,
        ]
    )

def train_transforms(image):
        return _train_transforms(image)
    # for item in examples:
        # item['pixel_values'] = _train_transforms(item['image'])
    # return examples

def val_transforms(image):
        return _val_transforms(image)
    # for item in examples:
        # item['pixel_values'] = _val_transforms(item['image'])
    # return examples

Size:  (224, 224)
Image mean:  [0.5, 0.5, 0.5]
Image std:  [0.5, 0.5, 0.5]


In [34]:
# from torch.utils.data import DataLoader
# from torch.utils.data import Dataset
# from pathlib import Path
# import torch
# import numpy as np
# from PIL import Image
# class MyDataset(Dataset):
#     def __init__(self, data_dir,test_frac=0.15,section="training",data_augmentation=False):
#         self.num_class = 0
#         self.test_frac = test_frac
#         self.data_augmentation=data_augmentation
#         self.section=section
#         self.transform=train_transforms if self.section=="training" else val_transforms
#         self.generate_data_list(data_dir)


#     def __len__(self):
#         return len(self.data)
    
#     def generate_data_list(self,data_dir):
#         # 类别名 [yes,no]
#         class_names = sorted(f"{x.name}" for x in Path(data_dir).iterdir() if x.is_dir())  # folder name as the class name
#         # 2
#         self.num_class = len(class_names)
#         image_files_list = []
#         image_class = []
#         # [[class1图片列表][class2图片列表]]
#         image_files = [[f"{x}" for x in (Path(data_dir) / class_names[i]).iterdir()] for i in range(self.num_class)]
#         num_each = [len(image_files[i]) for i in range(self.num_class)]
#         # 
#         max_value=max(num_each)
#         enlarge_factor=[max_value//num_each[i] for i in range(self.num_class)]
#         if not self.data_augmentation:
#             enlarge_factor=[1]*self.num_class
#         print('this is the enlarge factor',enlarge_factor)

#         class_name = []
#         # 读取所有图片为一个二维list [[class1图片列表][class2图片列表]]
#         # 对于每一类图片
#         for i in range(self.num_class):
#             # 将图片列表合并 [[class1图片列表][class2图片列表]] -> [class1图片列表+class2图片列表]
#             image_files_list.extend(image_files[i]*enlarge_factor[i])
#             # 为每个图片标记类别，类别标签从0开始，记录index [0,0,0,1,1,1]
#             image_class.extend([i] * num_each[i]*enlarge_factor[i])
#             # 为每个图片标记类别名 [yes,yes,yes,no,no,no]
#             class_name.extend([class_names[i]] * num_each[i]*enlarge_factor[i])
#         length = len(image_files_list)
#         # 生成图片索引 [0,1,2,3,4,5]
#         indices = np.arange(length)
#         # 打乱图片顺序
#         np.random.shuffle(indices)
#         test_length = int(length * self.test_frac)
#         if self.section == "test":
#             section_indices = indices[:test_length]
#         elif self.section == "training":
#             section_indices = indices[test_length:]
#         else:
#             raise ValueError(
#                 f'Unsupported section: {self.section}, available options are ["training", "validation", "test"].'
#             )
#         def convert_image(image_path):
#             image = Image.open(image_path)
#             if image.mode != 'RGB':
#                 image=image.convert('RGB')
#             return image
#         self.data=[{"image":convert_image(image_files_list[i]),"label": image_class[i]}  for i in section_indices ]
#         self.data=self.transform(self.data)
    
#     def __getitem__(self, index):
#         # return self.data[index]
#         img=self.data[index]["pixel_values"]
#         label=self.data[index]['label']
#         return {'pixel_values':img,'label':label}
#         # return img,label

In [35]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import os
from pathlib import Path
import torch
import numpy as np
from PIL import Image

class MyDataset(Dataset):
    def __init__(self, data_dir,test_frac=0.15,section="training",balance=False):
        self.num_class = 0
        self.test_frac = test_frac
        self.section=section
        self.transform=train_transforms if self.section=="training" else val_transforms
        self.generate_data_list(data_dir)
        if balance:
            self.balance_classes()


    def __len__(self):
        return len(self.samples)
    #
    def balance_classes(self):
        from collections import Counter
        # 3 暂时不分类
        label2_counter=Counter(x[2] for x in self.samples)
        print("this is label2 counter: ", label2_counter)
        max_label2_count = max(label2_counter.values())
        print("max label2 count: ", max_label2_count)
        # before balance
        print("total num before first balance: ", len(self.samples))
        print("Deep: ", label2_counter[0])
        print("Lobar: ", label2_counter[1])
        print("Subtentorial: ", label2_counter[2])
        # # 为了平衡类别，复制少数类的数据，先复制label2
        balanced_samples = []
        for key in label2_counter.keys():
            factor = max_label2_count // label2_counter[key]
            if key==3:
                factor=1
            for i in range(len(self.samples)):
                if self.samples[i][2] == key:
                    balanced_samples.extend([self.samples[i]]*factor)
        
        self.samples = balanced_samples
        print("total num after first balance: ", len(self.samples))
        label2_counter=Counter(x[2] for x in self.samples)
        print("counter after first balance: ", label2_counter)
        balanced_samples=[]
        label1_counter = Counter(x[1] for x in self.samples)
        print("total num before second balance: ", len(self.samples))
        print("no tumor: ", label1_counter[0])
        print("tumor: ", label1_counter[1])
        max_label1_count = max(label1_counter.values())
        print("max label1 count: ", max_label1_count)
        for label in label1_counter.keys():
            factor = max_label1_count // label1_counter[label]
        #     # 复制因子次每个类别的数据
            for i in range(len(self.samples)):
                if self.samples[i][1] == label:
                    balanced_samples.extend([self.samples[i]]* factor)
        self.samples = balanced_samples
        print("total num after balance: ", len(self.samples))
        label1_counter = Counter(x[1] for x in self.samples)
        print("no tumor: ", label1_counter[0])
        print("tumor: ", label1_counter[1])


    def generate_data_list(self,data_dir):
        # 类别名 [yes,no]
        # class_names = sorted(f"{x.name}" for x in Path(data_dir).iterdir() if x.is_dir())  # folder name as the class name
        no_tumor_dir = os.path.join(data_dir, 'no')
        no_tumor_images = [(os.path.join(no_tumor_dir, img), 0, 3) for img in os.listdir(no_tumor_dir)]
        yes_tumor_dir = os.path.join(data_dir, 'yes')
        tumor_classes = {'Deep': 0, 'Lobar': 1, 'Subtentorial': 2}
        yes_tumor_images = []
        for tumor_class, label in tumor_classes.items():
            class_dir = os.path.join(yes_tumor_dir, tumor_class)
            yes_tumor_images += [(os.path.join(class_dir, img), 1, label) for img in os.listdir(class_dir)]
        self.samples = no_tumor_images + yes_tumor_images
        # self.data=self.transform(self.data)
    
    def __getitem__(self, index):
        img_path, has_tumor, tumor_type = self.samples[index]
        image = Image.open(img_path).convert('RGB')
        image=self.transform(image)
        #filename=self.data[index]['file_name']
        # # return {'pixel_values':img,'label':label}
        # return img,label,filename
        return {'pixel_values':image,'label1':has_tumor,'label2':tumor_type}
        # return image,has_tumor, tumor_type


In [36]:
from torch.utils.data import random_split
data_dir='/home/jialiangfan/DTViT/dataset/'
dataset=MyDataset(data_dir)
train_size = int(0.8 * len(dataset))
val_size = int(0.1*len(dataset))
test_size=len(dataset) - train_size-val_size
train_dataset, val_dataset,test_dataset = random_split(dataset, [train_size,val_size,test_size])
# train_dataset = MyDataset(data_dir,test_frac=0.15,section="training",data_augmentation=True)
# test_dataset=MyDataset(data_dir,test_frac=0.15,section="test",data_augmentation=True)

In [43]:
len(train_dataset),len((test_dataset)),len(val_dataset)
# 155+98=253

(10120, 1266, 1265)

In [44]:
# def collate_fn(examples):
#     pixel_values = torch.stack([example["pixel_values"] for example in examples])
#     labels = torch.tensor([example["label"] for example in examples])
#     return {"pixel_values": pixel_values, "labels": labels}


In [88]:
train_dataloader = DataLoader(train_dataset,batch_size=16)
test_dataloader = DataLoader(test_dataset,batch_size=4)

In [89]:
batch = next(iter(train_dataloader))
print(batch.keys())
# batch['pixel_values'].shape,batch['label'].shape

# print(batch['pixel_values'].shape)
# for k,v in batch.items():
  # if isinstance(v, torch.Tensor):
    # print(k, v.shape)

dict_keys(['pixel_values', 'label1', 'label2'])


In [90]:
batch = next(iter(test_dataloader))
for k,v in batch.items():
  if isinstance(v, torch.Tensor):
    print(k, v.shape)

pixel_values torch.Size([4, 3, 224, 224])
label1 torch.Size([4])
label2 torch.Size([4])


Of course, we would like to know the actual class name, rather than the 

---

integer index. We can obtain that by creating a dictionary which maps between integer indices and actual class names (id2label):

## Preprocessing the data

We will now preprocess the data. The model requires 2 things: `pixel_values` and `labels`. 

We will perform data augmentaton **on-the-fly** using HuggingFace Datasets' `set_transform` method (docs can be found [here](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=set_transform#datasets.Dataset.set_transform)). This method is kind of a lazy `map`: the transform is only applied when examples are accessed. This is convenient for tokenizing or padding text, or augmenting images at training time for example, as we will do here. 

It's very easy to create a corresponding PyTorch DataLoader, like so:

## Define the model

Here we define the model. We define a `ViTForImageClassification`, which places a linear layer ([nn.Linear](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)) on top of a pre-trained `ViTModel`. The linear layer is placed on top of the last hidden state of the [CLS] token, which serves as a good representation of an entire image. 

The model itself is pre-trained on ImageNet-21k, a dataset of 14 million labeled images. You can find all info of the model we are going to use [here](https://huggingface.co/google/vit-base-patch16-224-in21k).

We also specify the number of output neurons by setting the id2label and label2id mapping, which we be added as attributes to the configuration of the model (which can be accessed as `model.config`).

In [91]:
from transformers.models.vit.modeling_vit import * 

_IMAGE_CLASS_CHECKPOINT = "google/vit-base-patch16-224"
_CONFIG_FOR_DOC = "ViTConfig"
_IMAGE_CLASS_EXPECTED_OUTPUT = "Egyptian cat"

class DualViTForImageClassification(ViTPreTrainedModel):
    def __init__(self, config: ViTConfig) -> None:
        super().__init__(config)

        self.num_labels = config.num_labels
        self.vit = ViTModel(config, add_pooling_layer=False)

        # Classifier head
        self.classifier1 = nn.Linear(config.hidden_size, 2) if config.num_labels > 0 else nn.Identity()
        self.classifier2 = nn.Linear(config.hidden_size, 4) if config.num_labels > 0 else nn.Identity()

        # Initialize weights and apply final processing
        self.post_init()

    @add_start_docstrings_to_model_forward(VIT_INPUTS_DOCSTRING)
    @add_code_sample_docstrings(
        checkpoint=_IMAGE_CLASS_CHECKPOINT,
        output_type=ImageClassifierOutput,
        config_class=_CONFIG_FOR_DOC,
        expected_output=_IMAGE_CLASS_EXPECTED_OUTPUT,
    )
    def forward(
        self,
        pixel_values: Optional[torch.Tensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        label1: Optional[torch.Tensor] = None,
        label2: Optional[torch.Tensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        interpolate_pos_encoding: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[tuple, ImageClassifierOutput]:
        r"""
        labels (`torch.LongTensor` of shape `(batch_size,)`, *optional*):
            Labels for computing the image classification/regression loss. Indices should be in `[0, ...,
            config.num_labels - 1]`. If `config.num_labels == 1` a regression loss is computed (Mean-Square loss), If
            `config.num_labels > 1` a classification loss is computed (Cross-Entropy).
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        # vit的输出是一个元组，第一个元素是最后一个token的输出，第二个元素是所有token的输出
        outputs = self.vit(
            pixel_values,
            head_mask=head_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            interpolate_pos_encoding=interpolate_pos_encoding,
            return_dict=return_dict,
        )
        sequence_output = outputs[0]
        logits1 = self.classifier1(sequence_output[:, 0, :])
        logits2 = self.classifier2(sequence_output[:, 0, :])
        loss = None
        loss_fct = CrossEntropyLoss()
        if label1 is not None:
            # move labels to correct device to enable model parallelism
            label1 = label1.to(logits1.device)
            loss1 = loss_fct(logits1.view(-1, self.num_labels), label1.view(-1))
        if label2 is not None:
            # move labels to correct device to enable model parallelism
            label2 = label2.to(logits2.device)
            loss2 = loss_fct(logits2.view(-1, self.num_labels), label2.view(-1))
        loss=loss1+loss2
        if not return_dict:
            output = (logits1,) + outputs[1:]
            return ((loss,) + output) if loss is not None else output

        return ImageClassifierOutput(
            loss=loss,
            logits=logits1,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )


In [92]:
from transformers import ViTConfig
from torch import nn
config=ViTConfig()
config.problem_type="single_label_classification"
model = DualViTForImageClassification.from_pretrained("google/vit-base-patch16-224-in21k",config=config)
# model.classifier=nn.Linear(in_features=768, out_features=train_dataset.num_class, bias=True)

loading weights file model.safetensors from cache at /home/jialiangfan/.cache/huggingface/hub/models--google--vit-base-patch16-224-in21k/snapshots/b4569560a39a0f1af58e3ddaf17facf20ab919b0/model.safetensors
Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing DualViTForImageClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing DualViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DualViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DualViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16

In [93]:
model.classifier1

Linear(in_features=768, out_features=2, bias=True)

In [94]:
model.classifier2

Linear(in_features=768, out_features=4, bias=True)

In [95]:
# for name, param in model.named_parameters():
#     if name.startswith("classifier"):
#         param.requires_grad = True
#     else:
#         model.requires_grad_=False

In [96]:

# 遍历模型中的所有参数
for name, param in model.named_parameters():
    # 检查参数是否被冻结
    if param.requires_grad:
        print(f"参数 {name} 没有被冻结，将会更新。")
    else:
        print(f"参数 {name} 被冻结，不会更新。")

参数 vit.embeddings.cls_token 没有被冻结，将会更新。
参数 vit.embeddings.position_embeddings 没有被冻结，将会更新。
参数 vit.embeddings.patch_embeddings.projection.weight 没有被冻结，将会更新。
参数 vit.embeddings.patch_embeddings.projection.bias 没有被冻结，将会更新。
参数 vit.encoder.layer.0.attention.attention.query.weight 没有被冻结，将会更新。
参数 vit.encoder.layer.0.attention.attention.query.bias 没有被冻结，将会更新。
参数 vit.encoder.layer.0.attention.attention.key.weight 没有被冻结，将会更新。
参数 vit.encoder.layer.0.attention.attention.key.bias 没有被冻结，将会更新。
参数 vit.encoder.layer.0.attention.attention.value.weight 没有被冻结，将会更新。
参数 vit.encoder.layer.0.attention.attention.value.bias 没有被冻结，将会更新。
参数 vit.encoder.layer.0.attention.output.dense.weight 没有被冻结，将会更新。
参数 vit.encoder.layer.0.attention.output.dense.bias 没有被冻结，将会更新。
参数 vit.encoder.layer.0.intermediate.dense.weight 没有被冻结，将会更新。
参数 vit.encoder.layer.0.intermediate.dense.bias 没有被冻结，将会更新。
参数 vit.encoder.layer.0.output.dense.weight 没有被冻结，将会更新。
参数 vit.encoder.layer.0.output.dense.bias 没有被冻结，将会更新。
参数 vit.encoder.layer.0.layer

To instantiate a `Trainer`, we will need to define three more things. The most important is the `TrainingArguments`, which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional.

We also set the argument "remove_unused_columns" to False, because otherwise the "img" column would be removed, which is required for the data transformations.

In [97]:
from transformers import TrainingArguments, Trainer
import os

os.environ['NCCL_P2P_DISABLE'] = '1'
os.environ['NCCL_IB_DISABLE'] = '1'

metric_name = "accuracy"
args = TrainingArguments(
    "Brain-Tumor-Detection",
    save_strategy="epoch",
    evaluation_strategy="epoch",
    learning_rate=2e-5, #0.00002, #0.00002
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    logging_dir='logs',
    remove_unused_columns=False,
    report_to="tensorboard",
)
# args.set_optimizer(name="sgd")

/home/jialiangfan/miniconda3/envs/medical/lib/python3.8/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
PyTorch: setting up devices


Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, set the training and evaluation batch_sizes and customize the number of epochs for training, as well as the weight decay.

We also define a `compute_metrics` function that will be used to compute metrics at evaluation. We use "accuracy" here.


In [98]:
from sklearn.metrics import accuracy_score
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return dict(accuracy=accuracy_score(predictions, labels))

In [99]:
import torch
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    # data_collator=collate_fn,
    compute_metrics=compute_metrics,
    tokenizer=processor,
    
)

## Train the model


In [100]:
import os
trainer.train()

***** Running training *****
  Num examples = 10,120
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Training with DataParallel so batch size has been adjusted to: 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 6,330
  Number of trainable parameters = 85,803,270


ValueError: Caught ValueError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/jialiangfan/miniconda3/envs/medical/lib/python3.8/site-packages/torch/nn/parallel/parallel_apply.py", line 83, in _worker
    output = module(*input, **kwargs)
  File "/home/jialiangfan/miniconda3/envs/medical/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1532, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/home/jialiangfan/miniconda3/envs/medical/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1541, in _call_impl
    return forward_call(*args, **kwargs)
  File "/tmp/ipykernel_2107629/1774055843.py", line 67, in forward
    loss2 = loss_fct(logits2.view(-1, self.num_labels), label2.view(-1))
  File "/home/jialiangfan/miniconda3/envs/medical/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1532, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/home/jialiangfan/miniconda3/envs/med

## Evaluation

Finally, let's evaluate the model on the test set:

In [ ]:
outputs = trainer.predict(test_dataset)


NameError: name 'trainer' is not defined

In [ ]:
print(outputs.metrics)

NameError: name 'outputs' is not defined